# Stock purchase recommendations with Machine Learning

In [2]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm # progress bar
#import fastparquet
#import zipline

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [3]:
pd.set_option('display.max_columns', 1500)

In [4]:
# read in the Quandl.com API key - saved in separate file to keep it private

filepath = '../confidential-API-key.txt'  
with open(filepath) as fp:  
   my_Quandl_API_key = fp.readline()

## Load stock data from Quandl

In [5]:
def load_stock_data(ticker, start_date, end_date, Quandl_API_key=my_Quandl_API_key):
    '''
    Downloads stock data from Quandl, drop some columns, resort datafram, and return
    Inputs:
        ticker - a stock ticker symbol (needs to be valid - no error checking implemented)
        start_date - first date of stock prices
        end_date - last date of stock prices
        Quandl_API_key - string with valid API key for Quandl.com data queries
    Outputs:
        stock_data - DataFrame with stock price data, sorted in ascending date order
    '''
    
    # download data from Quandl with Pandas Datareader
    stock_data = web.DataReader(name=symbol, data_source='quandl', start=start_date, end=end_date, access_key=Quandl_API_key)
    
    # need ascending index for the TA-lib indicators to work properly
    stock_data.sort_index(inplace=True)
    
    # keep only the columns with adjusted data to eliminate any issues due to stock splits
    stock_data = stock_data[['AdjVolume', 'AdjOpen', 'AdjHigh', 'AdjLow', 'AdjClose']]
    
    # DataFram has a second level column index with the stock ticker - not needed so drop it
    stock_data.columns = stock_data.columns.droplevel(1)
    
    # add row index (highest number is most recent date) - this will be used to later restack the rows for feature matrix
    stock_data['row_index'] = range(0, stock_data.shape[0])
    
    return stock_data    

In [6]:
# test loading stock data:

start = '2013-01-01'
end = '2019-04-06'
symbol = ['AAPL']

df = load_stock_data(symbol, start, end)

df.head()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index
Date,,,,,,
2013-01-02,140129500.0,71.816894,71.969911,70.236149,71.195748,0
2013-01-03,88241300.0,71.046621,71.278740,70.154453,70.296565,1
2013-01-04,148583400.0,69.631213,69.847109,68.187094,68.338996,2
2013-01-07,121039100.0,67.690619,68.637250,66.808825,67.937002,3
2013-01-08,114676800.0,68.625579,68.973109,67.593362,68.119845,4


In [ ]:
df.shape

In [ ]:
df.tail()

## Define Target column: profitability of trade

The goal is the define a column "setup_for_profitable_trade" based on a trading strategy. The data will be analyzed for this trade setup:

Run the analysis in the evening after market close. That day's row of market data will then be either deemed as profitable or not for a trade to be initiated and closed in the following days. This setup of the data ensures no lookahead is happening.

## Trading Strategy:

1. After market close on day N decide if buying stock at market Open on day N+1
2. Open position: submit market order for day N+1 prior to market open
3. Close position: submit sell order for market open for day N+10

Threshold to decide to buy the stock: expected profit from open to close: 2.5% profit

Store in row N if this trade was deemed profitable.

In [ ]:
def add_flag_for_profitable_trade_setups(df, delete_interim_calculation_cols=True):
    '''
    Adds column to dataframe that identifies profitable trading setup
    Inputs:
        df - dataframe with stock data
        delete_interim_calculation_cols - if false: keep the interim calculations - good for debugging
    Outputs:
        df - dataframe with additional columns
    
    '''
    
    # trade strategy: after market close on day N, set a buy at market open on day N+1 and sell at market close on day N+1
    # for expected gain of at least 2.5%
    profitability_threshold = 0.025

    # use helper columns to calculate profit
    df['strategy_open_price'] = df['AdjOpen'].shift(-1) # AdjOpen from day N+1
    df['strategy_close_price'] = df['AdjOpen'].shift(-10) # AdjOpen from day N+10
    df['strategy_profit_dollars'] = df['strategy_close_price'] - df['strategy_open_price']
    df['strategy_profitability'] = df['strategy_profit_dollars'] / df['strategy_open_price']

    # use categorical field to encode "setup_for_succesful_trade": 1=yes, 0=no
    df['setup_for_profitable_trade'] = df['strategy_profitability'] >= profitability_threshold
    
    if delete_interim_calculation_cols:
        df = df.drop(columns=['strategy_open_price', 'strategy_close_price', 'strategy_profit_dollars', 'strategy_profitability'])
    
    # drop any rows that have NaNs in them. especially the last few rows will have NaNs because the profitability calc
    # looks into the future beyond the last row. these rows need to get deleted - the profitability is not defined there
    df = df.loc[df.notnull().all(axis=1), :]
    
    return df

In [ ]:
# testing: show calculations:
df = add_flag_for_profitable_trade_setups(df, False)
df.tail(10)

In [ ]:
# without verbose results:
df = add_flag_for_profitable_trade_setups(df)
df.tail(10)

In [ ]:
df['setup_for_profitable_trade'].sum()

## Feature Engineering

In [ ]:
#split df into features and target (column setup_for_profitable_trade)
df_X_base_data = df.drop(columns=['setup_for_profitable_trade'])

### Feature Engineering - Add Technical Analysis Indicators

In [ ]:
def add_TALib_indicator(df, attribute, indicator_func, *args):
    '''
    Adds a column to a dataframe:
        column name is the name of the technical indicator as specified by indicator_func
        column content is the function calculated on the attribute column
    Example: add_TALib_indicator(df, 'AdjClose', ta.RSI, 14) creates a new column called RSI with 
             the 14 day RSI of the values of the column 'AdjClose'
    Inputs:
        df - dataframe - needs to be sorted in date ascending order
        attribute - column name to be used in TA-Lib calculation
        indicator_func - name of a TA-Lib function
        *args - optional parameters for indicator_func
        
    Oupputs:
        df - datarame with new column added
        func_name - name of the new colunm
    
    '''
    # get the name of the indicator from TA-Lib
    func_name = attribute + indicator_func.__name__ + str(*args)
    
    # add new column, calculated based on attribute column
    df.loc[:, func_name] = indicator_func(df.loc[:, attribute].values, *args)
    
    return df, func_name

In [ ]:
def add_comparison_cols_for_indicator(df, base_col_name, indicator_col_name, delete_indicator_col=True):
    '''
    adds columns that compare indicator_col to base_col: ratio, crossover, above/below
    Inputs:
        df - dataframe
        base_col_name - name of column that the indicator will get compared to
        indicator_col_name - name of column that has indicator values
        delete_base_col - yes/no on if to keep the base col or not
    Output:
        df - modified df with added & removed columns
    '''
   
    # indicator to base column ratio:
    df.loc[:, indicator_col_name + '_to_' + base_col_name + '_ratio'] = df.loc[:, indicator_col_name] / df.loc[:, base_col_name]
    
    # base col above indicator:
    base_above_indicator_col_name = base_col_name + '_above_' + indicator_col_name
    df.loc[:, base_above_indicator_col_name] = df.loc[:, indicator_col_name] < df.loc[:, base_col_name]
    
    # did base cross indicator
    base_crossed_indicator_col_name = base_col_name + '_crossed_' + indicator_col_name
    df.loc[:, base_crossed_indicator_col_name] = df.loc[:, base_above_indicator_col_name] != df.loc[:, base_above_indicator_col_name].shift(1)
    
    if delete_indicator_col:
        df = df.drop(columns=indicator_col_name)
    
    return df

In [ ]:
df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.RSI, 14)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=False)

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 10)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=False)

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 50)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=False)

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 200)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=False)


df_X_base_data.tail()

In [ ]:
df_X_base_data.head(16)
#confirms NaN for RSI on top for first 14 dates since it is a 14 day RSI

### Feature Engineering - Changing from actual values to percentage changes

In [ ]:
def feat_eng_changes_values_to_change(df, cols_set_vals_to_change, delete_original_cols=True):
    '''
    Instead of the actual values in some columns, we care about the change from one day to the next.
    This function calculates that change for the given columns and then either keeps or drops (default) the origianl columns
    Input:
        df - a dataframe
        cols_set_vals_to_change - names of columns to work on.
        delete_original_cols - keep or delete original columns
    Output:
        df - dataframe with new columns added. the value in row N is now the change from row N-1 to row N (instead of the actual values)
    '''    

    # calculate the change from row N-1 to row N
    df_chg_cols = (df[cols_set_vals_to_change] / df[cols_set_vals_to_change].shift(1) - 1)

    # add suffix to the column names
    df_chg_cols = df_chg_cols.add_suffix('_chg')

    # join the data onto the original data fram
    df = df.join(df_chg_cols)

    if delete_original_cols:
        # drop the original columns
        df = df.drop(columns=cols_set_vals_to_change)
        
    return df

cols_set_vals_to_change = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']
df_X_base_data = feat_eng_changes_values_to_change(df_X_base_data, cols_set_vals_to_change, delete_original_cols=False)

df_X_base_data.tail()

### Feature Engineering - Reshaping the prior day data into ML-ready features matrix
#### --- build pieces for the dataframe section unpivoting ---
Goal: pull out n_features rows, flatten, and build meaningful column names that indicate how many days' back the data is from

In [ ]:
# parameter for how many days of history to include in feature list
n_days_features = 5

# total lenght of df
n_data_points = df.shape[0]

In [ ]:
i = 505
# pull out n_days_features of rows from current position
df_extract = df_X_base_data.iloc[i-n_days_features:i, :].copy()

# change the index the be "days into the past" - eg current day is 0, prior day is -1, ...
df_extract.loc[:, 'row_index'] = range(-n_days_features+1, 1)

# make this the new index
df_extract.set_index('row_index', inplace=True)

df_extract

In [ ]:
# normalize columns from dollars to "1" - turns the columns into ratios compared to day N (index 0)

cols_to_normalize_to_1_for_day_0 = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']

df_extract.loc[:, cols_to_normalize_to_1_for_day_0] = df_extract.loc[:, cols_to_normalize_to_1_for_day_0] / df_extract.loc[0, cols_to_normalize_to_1_for_day_0]

df_extract

In [ ]:
# unstack and make it tall (ie unpivot)
df_extract = df_extract.unstack().reset_index()
df_extract

In [ ]:
# create new column with combined field names of attribute and index
# eg: AdjClose-1 for the adjusted close of day N-1 or AdjHigh-4 for the adjusted High of day N-4
df_extract['Attribute-index'] = df_extract['Attributes'] + df_extract['row_index'].apply(str)
# then drop Attributes and row_index columns since they are not needed anymore
df_extract.drop(columns=['Attributes', 'row_index'], inplace=True)
df_extract

In [ ]:
# set index one and transpose
target_row = df_extract.set_index('Attribute-index').T
# we now have one row of data that represents the prior n_feature_days worth of data:
target_row

In [ ]:
df_X_base_data.index.to_list()[504]

In [ ]:
target_row['Index'] = df_X_base_data.index.to_list()[504]
target_row = target_row.set_index('Index')
target_row

#### --- done building the pieces, now implement in one loop to cycle through all rows of df ---

In [ ]:
def create_feature_cols_df(df_X_base_data,
                           n_days_features=n_days_features,
                           cols_to_normalize_to_1_for_day_0 = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']):
    '''
    Take dataframe with date index (sorted increasing time) with multiple columns and return a new wider dataframe
    where the rows for the last n_days_features have been pivoted into additional columns
    Input:
        df_X_base_data - dataframe with date index
        n_days_features - number of prior days that are pivoted into the rows
        cols_to_normalize_to_1_for_day_0 - columns that will be normalized for day N. eg prices, volumes
    Output:
        df_X - datafram that has length of df_X_base_data.shape[0] - n_days_features and more columns than df_X_base_data
    '''
    
    df_X = pd.DataFrame()
    
    # total lenght of df
    n_data_points = df_X_base_data.shape[0]
    
    # cycle through each row of df, start at n_days_features-1 because we wouldn't have enough history for first rows
    for i in tqdm(range(n_days_features, n_data_points+1), desc='reshaping data into feature rows'):
        # i contains the rows number of df

        df_extract = df_X_base_data.iloc[i-n_days_features:i, :]

        # pull out n_days_features of rows from current position
        df_extract = df_X_base_data.iloc[i-n_days_features:i, :].copy()

        # change the index to be "days into the past" - eg current day is 0, prior day is -1, ...
        df_extract.loc[:, 'row_index'] = range(-n_days_features+1, 1)

        # make this the new index
        df_extract.set_index('row_index', inplace=True)
        
        # normalize columns from dollars to "1" - turns the columns into ratios compared to day N (index 0)
        df_extract.loc[:, cols_to_normalize_to_1_for_day_0] = df_extract.loc[:, cols_to_normalize_to_1_for_day_0] / df_extract.loc[0, cols_to_normalize_to_1_for_day_0]

        # unstack and make it tall (ie unpivot)
        df_extract = df_extract.unstack().reset_index()

        # create new column with combined field names of attribute and index
        # eg: AdjClose_-1 for the adjusted close of day N-1 or AdjHigh_-4 for the adjusted High of day N-4
        df_extract['Attribute-index'] = df_extract['Attributes'] + '_' + df_extract['row_index'].apply(str)
        # then drop Attributes and row_index columns since they are not needed anymore
        df_extract.drop(columns=['Attributes', 'row_index'], inplace=True)

        # set index one and transpose
        target_row = df_extract.set_index('Attribute-index').T
        # we now have one row of data that represents the prior n_feature_days worth of data

        # fill in the target_row index with the date from the index of the source dataframe df_X_base_data (ie, day N)
        target_row['Index'] = df_X_base_data.index.to_list()[i-1] # zero-indexed so need minus 1
        target_row = target_row.set_index('Index')

        df_X = df_X.append(target_row)
        
    return df_X

In [ ]:
# split df into features and target (column setup_for_profitable_trade)
df_X_base_data = df.drop(columns=['setup_for_profitable_trade'])
df_y = df['setup_for_profitable_trade']

# add technical indicators

# RSI
df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.RSI, 14)
# add threshold columns for above 80 and below 20
df_X_base_data.loc[:, 'RSI_above_80'] = df_X_base_data.loc[:, indicator_name] > 80
df_X_base_data.loc[:, 'RSI_below_20'] = df_X_base_data.loc[:, indicator_name] < 20
# normalize to values between 0 and 1
df_X_base_data.loc[:, indicator_name] = df_X_base_data.loc[:, indicator_name] / 100

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 10)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=True)

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 50)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=True)

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 200)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=True)

# feature engineering: instead of dollars/absolute values, calculate change from one day to next
cols_set_vals_to_change = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']
df_X_base_data = feat_eng_changes_values_to_change(df_X_base_data, cols_set_vals_to_change, delete_original_cols=False)

# create wide features matrix that includes prior days' data as columns
df_X = create_feature_cols_df(df_X_base_data,
                              n_days_features= 20,
                              cols_to_normalize_to_1_for_day_0 = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose'])

df_X.tail()

In [ ]:
df_X.shape

In [ ]:
fastparquet.write('./data/interim/df_X.parq', df_X)

In [ ]:
df_X = fastparquet.ParquetFile('./data/interim/df_X.parq').to_pandas()
df_X.tail()

### Feature Engineering - Date information

In [ ]:
def feat_eng_append_date_index_content(df):
    '''
    Assumes that the df index is date-time. Bolts on additional columns about the date
    '''

    df['year'] = df.index.year.values
    df['month'] = df.index.month.values
    df['week'] = df.index.week.values
    df['weekday'] = df.index.weekday.values
    df['day'] = df.index.day.values
    df['year'] = df.index.year.values
    df['year'] = df.index.year.values
    
    return df
    
df_X = feat_eng_append_date_index_content(df_X)
df_X.tail()

##### just testing aroung: can also use only one or a few columns and then join the dataframes back together

In [ ]:
#df_XO = create_feature_cols_df(df_X_base_data[['AdjOpen']])
#df_XO.tail()

In [ ]:
#df_X.join(df_XO).tail()

## Machine Learning

### Build Model

In [ ]:
# need to remove first n_days_features rows - they were eliminated during build of df_X and sizes need to match
df_y = df_y.iloc[n_days_features-1:]

In [ ]:
# remove all rows that have any NaNs in them - they come from technical indicators or the reshaping and we just don't have 
# any good strategy for imputation other than starting with more time series data
df_X = df_X.loc[df_X.notnull().all(axis=1), :]
df_X.shape

In [ ]:
# now we need to keep the same rows that we kept in df_X in df_y. We'll use the index to filter
df_y = df_y.loc[df_X.index]
df_y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, shuffle=False)

In [ ]:
pipeline = Pipeline([
    ('standardScaler', StandardScaler()),
    ('randomForest', RandomForestClassifier())
])
pipeline.get_params()

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred, labels=[False, True])

In [ ]:
def print_feature_importances(estimator):

    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_train.shape[1]):
        print(str(f + 1) + " importance: " + str(importances[indices[f]]) + ". feature name: " + X_train.columns[indices[f]])
        
print_feature_importances(pipeline.named_steps['randomForest'])

### Improve RandomForest model

In [ ]:
parameters = {
    'randomForest__min_samples_leaf': [1, 2, 5, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, scoring='f1', n_jobs=-1, verbose=6)
cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

In [ ]:
print_feature_importances(cv.best_estimator_.named_steps['randomForest'])

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred, labels=[False, True])

In [ ]:
X_test.join(y_test)

## next: add backtesting

## use TPOT
## add column for stock
## add additional technical indicators
## add crossover metrics